#### Rohan Bhatt, Shubhang Srikoti 
##### MSML605 -  Investigating the Impact of Storage Formats
Problem statement: How does the choice of storage format (CSV, Parquet, HDF5) impact the overall performance of a machine learning pipeline and its processes (data ingestion, memory overhead, time-to-train, and more).

NOTE: THIS NOTEBOOK IS THE 2GB VARIANT OF THE PARQUET




In [ ]:
#script to convert 10gb parquet to 2gb parquet
import pyarrow.parquet as pq, pyarrow as pa, math
from pathlib import Path

SRC = Path("data.parquet")          # 16-GB file
DST = Path(r"E:\ML2\data_2gb.parquet")

pq_src   = pq.ParquetFile(SRC)
n_rg     = pq_src.num_row_groups

# gather row-group sizes (compressed bytes on disk)
rg_sizes = [pq_src.metadata.row_group(i).total_byte_size for i in range(n_rg)]

target_bytes = 2 * 1024**3          # 2 GB
keep_rg      = []
cum          = 0
for i, sz in enumerate(rg_sizes):
    if cum + sz > target_bytes:
        break
    keep_rg.append(i)
    cum += sz

print(f"Keeping {len(keep_rg)} row-groups  →  ~{cum/1024**3:.2f} GB")

# read & write subset
tables = [pq_src.read_row_group(i) for i in keep_rg]
subset = pa.concat_tables(tables)
pq.write_table(subset, DST, compression="snappy")   # or "zstd"

print(f"Subset rows: {subset.num_rows:,}")
print("Wrote:", DST)

Keeping 169 row-groups  →  ~0.00 GB
